# Application: Learning optimal restrictions in a continuous-action game

## Setup and definitions

### Imports

In [5]:
import os, sys
sys.path.append(f'{os.getcwd()}/../../')

In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import itertools
import networkx as nx
import pandas as pd

from gymnasium.spaces import Discrete, Box, Space

from drama.restrictions import DiscreteVectorRestriction
from drama.wrapper import RestrictionWrapper
from drama.restrictors import Restrictor, RestrictorActionSpace, DiscreteVectorActionSpace
from drama.utils import flatdim, flatten, unflatten

from examples.utils import play, ReplayBuffer
from examples.traffic.env import TrafficEnvironment
from examples.traffic.agent import TrafficAgent
from examples.traffic.restrictor import TrafficRestrictor

from examples.traffic.utils import create_graph, analyze_graph, edge_path_to_node_path

### Definitions

In [7]:
graph = create_graph([
    ((0, 1), (0, 8, 1)), 
    ((0, 2), (11, 0, 0)), 
    ((1, 2), (1, 0, 0)), 
    ((1, 3), (11, 0, 0)), 
    ((2, 3), (0, 8, 1))
])

In [8]:
possible_agent_routes = [(0, 3)]

In [13]:
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

number_of_nodes = graph.number_of_nodes()
number_of_edges = graph.number_of_edges()

edge_list = list(graph.edges)
edge_indices = {e: i for i, e in enumerate(edge_list)}
edge_latencies = {i: graph[s][t]["latency"] for i, [s, t] in enumerate(edge_list)}

minimum_node_set = set(sum(possible_agent_routes, tuple()))

valid_edge_restrictions = []
for allowed_edges in powerset(range(number_of_edges)):
    subgraph = graph.edge_subgraph(edge_list[i] for i in allowed_edges)
    if minimum_node_set.issubset(subgraph.nodes) and all(nx.has_path(subgraph, s, t) for s, t in possible_agent_routes):
        valid_edge_restrictions.append(set(allowed_edges))

route_list = [tuple(edge_indices[e] for e in path)
        for s, t in possible_agent_routes
        for path in nx.all_simple_edge_paths(graph, s, t)
]
number_of_routes = len(route_list)

routes = {
        (s, t): [
            tuple(edge_indices[e] for e in path)
            for path in nx.all_simple_edge_paths(graph, s, t)
        ]
        for s in range(number_of_nodes)
        for t in range(number_of_nodes)
        if s != t
    }
route_indices = {tuple(r): i for i, r in enumerate(route_list)}

valid_route_restrictions = [np.array([set(route).issubset(edge_restriction) for route in route_list]) for edge_restriction in valid_edge_restrictions]

source_target_map = [(s, t) for s, t in possible_agent_routes for _ in nx.all_simple_edge_paths(graph, s, t)]

## Execution

### Test: Without Governance

In [15]:
number_of_agents = 2

agents = {f'agent_{i}': TrafficAgent(routes, source_target_map) for i in range (number_of_agents)}
env = TrafficEnvironment(graph, list(agents), possible_routes, number_of_steps=100)
policies = {id: agent.act for id, agent in agents.items()}

trajectory = play(env, policies, max_iter=50, verbose=False, record_trajectory=True, render_mode=None)

NameError: name 'possible_routes' is not defined

In [ ]:
trajectory.groupby('agent')['reward'].plot(legend=True, xlabel='Time step', ylabel='Reward');

In [ ]:
ax, *_ = trajectory.groupby('agent')['action'].plot(style='.', legend=True)
ax.set_yticks(list(route_indices.values()), [edge_path_to_node_path(route, edge_list) for route in route_indices.keys()]);
ax.set_ylabel('Route taken')
ax.set_xlabel('Time step')

### With governance

In [16]:
number_of_agents = 2

edge_list, edge_indices, edge_latencies, routes, route_list, route_indices = analyze_graph(graph)
number_of_edges = graph.number_of_edges()

agents = {f'agent_{i}': TrafficAgent(routes, route_indices, edge_indices) for i in range (number_of_agents)}
env = TrafficEnvironment(graph, list(agents), possible_routes, number_of_steps=100)

restrictor = TrafficRestrictor(Box(0, np.inf, shape=(number_of_edges, )), DiscreteVectorActionSpace(Discrete(len(routes))))
wrapper = RestrictionWrapper(env, restrictor, restrictor_reward_fns={'restrictor_0': lambda env, rewards: rewards[env.agent_selection]})

policies = {**{id: agent.act for id, agent in agents.items()}, 'restrictor_0': restrictor.act}

trajectory = play(wrapper, policies, max_iter=50, verbose=False, record_trajectory=True, render_mode=None)

TypeError: analyze_graph() missing 1 required positional argument: 'possible_agent_routes'

In [ ]:
trajectory.groupby('agent')['reward'].plot(legend=True, xlabel='Time step', ylabel='Reward');

In [ ]:
ax, *_ = trajectory[trajectory['agent'] != 'restrictor_0'].groupby('agent')['action'].plot(style='.', legend=True)
ax.set_yticks(list(route_indices.values()), [edge_path_to_node_path(route, edge_list) for route in route_indices.keys()]);
ax.set_ylabel('Route taken')
ax.set_xlabel('Time step')

## With self-learning restrictor

In [ ]:
number_of_agents = 2

total_timesteps = 500_000

number_of_edges = graph.number_of_edges()

agents = {f'agent_{i}': TrafficAgent(route_list, source_target_map) for i in range (number_of_agents)}
restrictor = TrafficRestrictor(number_of_edges, number_of_routes,
                               valid_route_restrictions, total_timesteps=total_timesteps)

env = TrafficEnvironment(graph, list(agents), possible_agent_routes, number_of_routes, edge_latencies, route_list, number_of_steps=100)

env = RestrictionWrapper(env, restrictor, return_object=True)

In [ ]:
restricted_history = pd.DataFrame(columns=['episode', 'episode_step', 'agent', 'observation', 'reward', 'action'], index=(range(total_timesteps)))
replay_buffer = ReplayBuffer(state_dim=flatdim(restrictor.observation_space), action_dim=flatdim(restrictor.action_space))

# Do not render during training
env.unwrapped.render_mode = None

current_timestep = 0
current_episode = 0
t = tqdm(total=total_timesteps)

while current_timestep < total_timesteps:
    env.reset()
    current_episode += 1
    current_episode_timestep = 0
    previous_restrictor_observation = None

    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()

        if agent == 'restrictor_0':
            if previous_restrictor_observation is not None:
                restrictor.learn(previous_restrictor_observation, previous_restrictor_action, observation, reward, termination or truncation)

            action = restrictor.act(observation)

            previous_restrictor_observation = observation
            previous_restrictor_action = action
        else:
            action = agents[agent].act(observation)

        if termination or truncation:
            action = None
        else:

        # print(f'{agent=}, {observation=}, {reward=}, {action=}')

            restricted_history.loc[current_timestep] = pd.Series({'episode': current_episode, 
                                               'episode_step': current_episode_timestep, 
                                               'agent': agent,
                                               'observation': observation, 
                                               'reward': reward, 
                                               'action': action}
                                               )
            
            current_timestep += 1
            current_episode_timestep += 1

        env.step(action)

        t.update()

In [ ]:
unrestricted_history = pd.DataFrame(columns=['episode', 'episode_step', 'agent', 'observation', 'reward', 'action'], index=(range(total_timesteps)))

# Do not render during training
env.unwrapped.render_mode = None

current_timestep = 0
current_episode = 0
t = tqdm(total=total_timesteps)

while current_timestep < total_timesteps:
    env.reset()
    current_episode += 1
    current_episode_timestep = 0
    previous_restrictor_observation = None

    for agent in env.agent_iter():
        observation, reward, termination, truncation, info = env.last()

        if agent == 'restrictor_0':
            action = 14
        else:
            action = agents[agent].act(observation)

        if termination or truncation:
            action = None
        else:
            unrestricted_history.loc[current_timestep] = pd.Series({'episode': current_episode, 
                                               'episode_step': current_episode_timestep, 
                                               'agent': agent,
                                               'observation': observation, 
                                               'reward': reward, 
                                               'action': action}
                                               )
            
            current_timestep += 1
            current_episode_timestep += 1

        env.step(action)

        t.update()

In [ ]:
def smoothen(data, kernel_size):
    kernel = np.ones(kernel_size) / kernel_size

    if data.ndim == 1:
        return np.convolve(data, kernel, mode='same')
    elif data.ndim == 2:
        return np.array([np.convolve(col, kernel, mode='same') for col in data.T]).T
    else:
        raise NotImplementedError

In [ ]:
restrictor_actions = restricted_history[restricted_history.agent == 'restrictor_0']['action'].astype(int)
one_hot_restrictor_actions = np.eye(len(valid_edge_restrictions))[restrictor_actions.to_numpy().reshape(-1)]

fig, ax = plt.subplots()

data = pd.DataFrame(smoothen(one_hot_restrictor_actions, kernel_size=5_000), index=restrictor_actions.index, columns=valid_edge_restrictions)
lines = ax.plot(data.iloc[5000:-5000], color='gray', lw=1)
# lines = ax.plot(restrictor_history.index, smoothen(one_hot_restrictor_actions, kernel_size=5_000), color='gray', lw=1)
lines[11].set_color('red')
ax.legend(labels=valid_edge_restrictions, loc='center left', bbox_to_anchor=(0.95, 0.5))
ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
# ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Frequency of restriction')
ax.set_xlabel('Time step')

fig.savefig('traffic-result-actions.pdf', bbox_inches='tight')

In [ ]:
unrestricted_social_welfare = unrestricted_history[unrestricted_history['agent'] == 'restrictor_0']['reward']
unrestricted_data = pd.Series(smoothen(unrestricted_social_welfare.to_numpy().squeeze(), kernel_size=1000), index=unrestricted_social_welfare.index)

restricted_social_welfare = restricted_history[restricted_history.agent == 'restrictor_0']['reward']
restricted_data = pd.Series(smoothen(restricted_social_welfare.to_numpy().squeeze(), kernel_size=1000), index=restricted_social_welfare.index)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3.5))
ax.plot(restricted_data[1000:-1000] / 2, lw=1, color='green', label='Restricted social welfare')
ax.plot(unrestricted_data[1000:-1000] / 2, lw=1, color='red', label='Unrestricted social welfare')

ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Reward')
ax.set_xlabel('Time step');
ax.legend(loc='upper left')
# ax.set_ylim(-1, -14)

fig.savefig('traffic-result-reward.pdf', bbox_inches='tight')


## Record multiple runs for mean/std plots

### Initialization

In [ ]:
restrictor_action_runs = pd.DataFrame(index=restricted_history.index[restricted_history['agent'] == 'restrictor_0'])
unrestricted_social_welfare_runs = pd.DataFrame(index=unrestricted_history.index[unrestricted_history['agent'] == 'restrictor_0'])
restricted_social_welfare_runs = pd.DataFrame(index=restricted_history.index[restricted_history['agent'] == 'restrictor_0'])

### Recording

#### Actions

In [ ]:
restrictor_actions = restricted_history[restricted_history.agent == 'restrictor_0']['action'].astype(int)
restrictor_action_runs = pd.concat([restrictor_action_runs, restrictor_actions], axis=1)

restrictor_action_runs.shape

#### Social welfare

In [ ]:
unrestricted_social_welfare = unrestricted_history[unrestricted_history['agent'] == 'restrictor_0']['reward']
restricted_social_welfare = restricted_history[unrestricted_history['agent'] == 'restrictor_0']['reward']
unrestricted_social_welfare_runs = pd.concat([unrestricted_social_welfare_runs, unrestricted_social_welfare], axis=1)
restricted_social_welfare_runs = pd.concat([restricted_social_welfare_runs, restricted_social_welfare], axis=1)

unrestricted_social_welfare_runs.shape, restricted_social_welfare_runs.shape

### Sanity check

In [ ]:
print(f'{restrictor_action_runs.shape=}, {unrestricted_social_welfare_runs.shape=}, {restricted_social_welfare_runs.shape=}')

### Analysis

#### Actions

In [ ]:
kernel_size = 5_000
one_hot_base = np.eye(len(valid_edge_restrictions))
essential_action = 11

one_hot_restrictor_action_runs = np.stack([one_hot_base[col] for col in restrictor_action_runs.to_numpy().T])

one_hot_restrictor_actions_mean = one_hot_restrictor_action_runs.mean(axis=0)
smooth_one_hot_restrictor_actions_mean = smoothen(one_hot_restrictor_actions_mean, kernel_size=kernel_size)
data_mean = pd.DataFrame(smooth_one_hot_restrictor_actions_mean, index=restrictor_actions.index)[kernel_size:-kernel_size]

one_hot_restrictor_actions_std = one_hot_restrictor_action_runs.std(axis=0)
smooth_one_hot_restrictor_actions_std = smoothen(one_hot_restrictor_actions_std, kernel_size=kernel_size)
data_std = pd.DataFrame(smooth_one_hot_restrictor_actions_std, index=restrictor_actions.index)[kernel_size:-kernel_size]

# Downsampling by factor 50
data_mean = data_mean[::50]
data_std = data_std[::50]

fig, ax = plt.subplots(figsize=((5.8, 4.5)))

lines = ax.plot(data_mean, color='gray', lw=1)
lines[essential_action].set_color('red')

for col in data_std:
    ax.fill_between(data_std.index, data_mean[col] - data_std[col], data_mean[col] + data_std[col], facecolor = 'gray' if int(col) != essential_action else 'red', alpha=0.1 if int(col) != essential_action else 0.2)

ax.legend(labels=valid_edge_restrictions, loc='center left', bbox_to_anchor=(0.95, 0.5))
ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Frequency of restriction')
ax.set_xlabel('Time step')

fig.savefig('traffic-result-actions.pdf', bbox_inches='tight')

#### Social welfare

In [ ]:
kernel_size = 1_000

unrestricted_social_welfare_mean = unrestricted_social_welfare_runs.mean(axis=1)
unrestricted_smooth_welfare_mean = pd.Series(smoothen(unrestricted_social_welfare_mean.to_numpy().squeeze(), kernel_size=kernel_size), index=unrestricted_social_welfare_mean.index)[kernel_size:-kernel_size].astype(float) / number_of_agents

unrestricted_social_welfare_std = unrestricted_social_welfare_runs.std(axis=1)
unrestricted_smooth_welfare_std = pd.Series(smoothen(unrestricted_social_welfare_std.to_numpy().squeeze(), kernel_size=kernel_size), index=unrestricted_social_welfare_std.index)[kernel_size:-kernel_size].astype(float) / number_of_agents

restricted_social_welfare_mean = restricted_social_welfare_runs.mean(axis=1)
restricted_smooth_welfare_mean = pd.Series(smoothen(restricted_social_welfare_mean.to_numpy().squeeze(), kernel_size=kernel_size), index=restricted_social_welfare_mean.index)[kernel_size:-kernel_size].astype(float) / number_of_agents

restricted_social_welfare_std = restricted_social_welfare_runs.std(axis=1)
restricted_smooth_welfare_std = pd.Series(smoothen(restricted_social_welfare_std.to_numpy().squeeze(), kernel_size=kernel_size), index=restricted_social_welfare_std.index)[kernel_size:-kernel_size].astype(float) / number_of_agents

# Downsampling by factor 50
unrestricted_smooth_welfare_mean = unrestricted_smooth_welfare_mean[::10]
unrestricted_smooth_welfare_std = unrestricted_smooth_welfare_std[::10]
restricted_smooth_welfare_mean = restricted_smooth_welfare_mean[::10]
restricted_smooth_welfare_std = restricted_smooth_welfare_std[::10]

fig, ax = plt.subplots(figsize=(7, 3.5))

ax.fill_between(unrestricted_smooth_welfare_std.index, unrestricted_smooth_welfare_mean - unrestricted_smooth_welfare_std, unrestricted_smooth_welfare_mean + unrestricted_smooth_welfare_std, facecolor='red', alpha=0.2)
ax.fill_between(restricted_smooth_welfare_std.index, restricted_smooth_welfare_mean - restricted_smooth_welfare_std, restricted_smooth_welfare_mean + restricted_smooth_welfare_std, facecolor='green', alpha=0.2)

ax.plot(unrestricted_smooth_welfare_mean, lw=1, color='red', label='Unrestricted social welfare')
ax.plot(restricted_smooth_welfare_mean, lw=1, color='green', label='Restricted social welfare')

ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Reward')
ax.set_xlabel('Time step');
ax.legend(loc='upper left')

fig.savefig('traffic-result-reward.pdf', bbox_inches='tight')